In [87]:
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Add
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping
import csv

from PIL import Image

import numpy as np
import time

In [37]:
#どんくらいで学習させるか関連x_test
num_sample=100 #画像の総フレーム数
timesteps=30 #一回のLSTMに入れる値の数
camera=5 #カメラの数

#画像関連
channels=3
img_width=100
img_height=100

# LSTMなどで用いる値
n_hidden = 256    # 出力次元
epochs = 100      # エポック数
batch_size = 5   # ミニバッチサイズ

In [38]:
class Data:
    def __init__(self, camera_num, file_name, samples, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=samples
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.learnData=[]
        self.labelData=[]
        
        self.make_selection()
        self.make_image()
    
    def make_selection(self):
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        one_hot = np.eye(self.camera_num)
        for row2 in csv_selection:
            self.labelData.append(one_hot[int(row2[1])])
    
    def make_image(self):
        for i in range(0, self.camera_num):
            img_list=[]
            print()
            for j in range(0,self.samples):
                #どのくらいデータロードが進んでいるか
                pro_size=20
                bar = int(j*pro_size/self.samples)
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(j / self.samples * 100.)
                print('\r{0}/{1} [{2}] {3}%'.format((i+1), self.camera_num, pro_bar, percent), end='')
                time.sleep(0.5)
                
                file_path = "image/"+str(i)+"/"+str(j)+".jpg"
                img = Image.open(file_path).convert('RGB') ## Gray->L, RGB->RGB
                img = img.resize((self.width, self.height))
                x = np.array(img, dtype=np.float32)
                x = x / 255.
                
                img_list.append(x)
            self.learnData.append(img_list)
    
    def get_learndata(self, num):
        test=[]
        for i in range(0,self.samples-self.timesteps):
            tmp_list=[]
            for j in range(0, self.timesteps):
                tmp_list.append(self.learnData[num][i+j])
            test.append(tmp_list)
        tmp=np.array(test)
        tmp=tmp.astype(np.float)
                
        return tmp
    
    def get_labeldata(self):
        test=[]
        for i in range(self.timesteps,self.samples):
            test.append(self.labelData[i])
        tmp = np.array(test)
        tmp = tmp.astype(np.float)
        
        return tmp
    
    def show(self):
        print()
        #print(self.get_learndata(0).shape)
        print(self.get_labeldata().shape)

In [39]:
data = Data(camera, "./", num_sample, timesteps, img_width, img_height)
data.show()


5/5 [=================== ] 99.000000%
(70, 5)


In [101]:
class Resnet18 :
    def __init__(self, width, height, channels):
        self.img_rows=height
        self.img_cols=width
        self.img_channels=channels
    
    def rescell(self, data, filters, kernel_size, option=False):
        strides=(1,1)
        if option:
            strides=(2,2)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        
        data=Convolution2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
        x=BatchNormalization()(x)
        x=Add()([x,data])
        x=Activation('relu')(x)
        
        return x
    
    def create_resnet18(self):
        inputs=Input(shape=(self.img_rows, self.img_cols, self.img_channels))
        x=Convolution2D(64,(7,7), padding="same", input_shape=(self.img_rows, self.img_cols),activation="relu")(inputs)
        x=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        
        x=self.rescell(x,128,(3,3),True)
        x=self.rescell(x,128,(3,3))

        x=self.rescell(x,256,(3,3),True)
        x=self.rescell(x,256,(3,3))
        
        x=self.rescell(x,512,(3,3),True)
        x=self.rescell(x,512,(3,3))
        
        model=Model(inputs=inputs,outputs=[x])
        return model

    def get_resnet18(self):
        return self.create_resnet18()

In [102]:
class Prediction :
    def __init__(self, timesteps, n_hidden, n_out, width, height):
        self.maxlen = timesteps
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.width=width
        self.height=height
        self.resnet = Resnet18(width, height, 3).get_resnet18()
        self.sharedLayer = self.create_sharedmodel()
    
    def create_sharedmodel(self):
        inputs = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        x = TimeDistributed(self.resnet,name="resnet")(inputs)
        x = TimeDistributed(GlobalAveragePooling2D(), name="GAP")(x)
        x = TimeDistributed(Dense(512), name="dense")(x)
        predictions = LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, 35),
             kernel_initializer = glorot_normal(seed=20181020),
             recurrent_initializer = orthogonal(gain=1.0, seed=20181020), 
             dropout = 0.01, 
             recurrent_dropout = 0.01)(x)
        
        shared_layers = Model(inputs, predictions, name="shared_layers")
        
        return shared_layers
    
    def create_model(self):
        model_input1 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input2 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input3 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input4 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input5 = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        mid_feature1 = self.sharedLayer(model_input1)
        mid_feature2 = self.sharedLayer(model_input2)
        mid_feature3 = self.sharedLayer(model_input3)
        mid_feature4 = self.sharedLayer(model_input4)
        mid_feature5 = self.sharedLayer(model_input5)
        
        merged_vector = keras.layers.concatenate([mid_feature1, mid_feature2, mid_feature3, mid_feature4, mid_feature5], axis=-1)
        
        mid_dense = Dense(self.n_hidden, activation='sigmoid')(merged_vector)
        predictions = Dense(self.n_out, activation='sigmoid')(mid_dense)
        
        model = Model(inputs=[model_input1, model_input2, model_input3, model_input4, model_input5], outputs=predictions)
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
    
    def train(self, x_train1, x_train2, x_train3, x_train4, x_train5, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=5, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train1, x_train2, x_train3, x_train4, x_train5], t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
              shuffle = True, callbacks = [early_stopping], validation_split = 0.3)
        return model, hist

In [103]:
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_102 (InputLayer)          (None, 30, 100, 100, 0                                            
__________________________________________________________________________________________________
input_103 (InputLayer)          (None, 30, 100, 100, 0                                            
__________________________________________________________________________________________________
input_104 (InputLayer)          (None, 30, 100, 100, 0                                            
__________________________________________________________________________________________________
input_105 (InputLayer)          (None, 30, 100, 100, 0                                            
__________________________________________________________________________________________________
input_106 

In [ ]:
# モデル定義
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
# 学習
model , hist = pred.train(data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4), 
                                data.get_labeldata(), batch_size, epochs)
# テスト
score = model.evaluate([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)], 
                                data.get_labeldata(), batch_size = batch_size, verbose = 1)
print("score:", score)

pre = [[0 for i in range(3)] for j in range(3)]
# 正答率集計
preds = model.predict([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)])
correct = 0

for i in range(len(preds)):
    pred = np.argmax(preds[i,:])
    tar = np.argmax(data.get_labeldata()[i,:])
    pre[pred][tar]+=1
    if pred == tar :
        correct += 1

print("正答率:", 1.0 * correct / len(preds))